In [1]:
#url: http://www.emigrazioneveneta.com/

In [1]:
## BIBLIOTECAS:
    # pip install requests2
    # pip install pandas
    # pip install lxml
    # pip install beautifulsoup4
    # pip install selenium

import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
import os
import datetime
import time
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import string

time1=datetime.datetime.now()

In [9]:
def build_driver(download_path): 
    
    '''
    Esta função inicializa o webdriver incluindo as preferências.
    
    args:
    download_path: diretório da pasta de download.
    
    '''

    print("Ligando o Chrome Remoto...")
    option = Options()
    option.headless = True
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized") #Maximizar
    prefs = {"profile.default_content_settings.popups": 0,
             "download.default_directory": download_path, # Importante-Finalizar com barra dupla
             "download.prompt_for_download": False,
             "directory_upgrade": True,
             "safebrowsing.enabled": True}
    options.add_experimental_option("prefs", prefs)
    #driver=webdriver.Firefox()
    #driver = webdriver.Chrome(options=options)
    driver = webdriver.Chrome(r"C:\Users\John Melati\Documents\GitHub\web_scraping_emigrazione_veneta\chromedriver.exe", options=options)
    print("Ok")
    return driver

def login(driver, url_login): 
    
    '''
    Esta função realiza o login no site.
    
    args:
    driver: webdriver.
    url_login: url do site.
    
    '''

    print("Entrando na página de login...")
    driver.get(url_login)
    time.sleep(2)
    driver.implicitly_wait(10)
    print("Ok")
   #print("Realizando Login...")
   #name = driver.find_element_by_name("Ecom_User_ID")
   #name.send_keys("jmelati")
   #print("Preenchimento de Login: Ok")
   #password = driver.find_element_by_name("Ecom_Password")
   #password.send_keys("D@0912emb56YdR10")
   #print("Preenchimento de Senha: Ok")
   #login_box = driver.find_element_by_class_name("send")
   #login_box.click()
   #print("Logon: Ok")
   #time.sleep(2)
   #driver.implicitly_wait(10)

def element_click_xpath(xpath):

    '''
    Esta função simula o click do mouse baseado no xpath do HTML.
    
    args:
    xpath: xpath do HTML.
    
    ''' 
    
    time.sleep(1)
    click_box = driver.find_element_by_xpath(xpath)
    click_box.click()
    driver.implicitly_wait(10)
    print("click ok")
    
def element_find_xpath(xpath):

    '''
    Esta função simula o click do mouse baseado no xpath do HTML.
    
    args:
    xpath: xpath do HTML.
    
    ''' 
    
    time.sleep(1)
    text = driver.find_element(by="xpath", value=xpath)
    driver.implicitly_wait(10)
    print("get text ok")
    return text.text

def filter_project(project):

    '''
    Esta função realiza o preenchimento da aba de filtro com o projeto da aeronave.
    
    args:
    project: E1 ou E2.
    
    '''  
    print("fill project")
    time.sleep(1)
    project_box = driver.find_element_by_xpath("//html/body/div[1]/main/div[2]/form/div/div[2]/input")
    
    project_box.send_keys(Keys.CONTROL + 'a', Keys.BACKSPACE) 
    
    project_box.send_keys(project)
    project_box.send_keys(Keys.RETURN)
    driver.implicitly_wait(10)
    print("ok")
    
def combinacao_3_caracteres(n):
    lista_caracteres = []
    for i in range(4,5,1):
        for j in range(0,n,1):
            for k in range(0,n,1):
                temp_name = (string.ascii_lowercase[i] + string.ascii_lowercase[j] + string.ascii_lowercase[k]) 
                lista_caracteres.append(temp_name)
    return lista_caracteres

def navegar_nas_paginas(texto, num_registros, df):
    for i in range(0,num_registros,20):
        print(i)
        url = "http://www.emigrazioneveneta.com/busca?searchword={}&searchphrase=all&limitstart={}".format(texto, i)
        driver.implicitly_wait(10)
        time.sleep(1)
        print(url)
        driver.get(url)
        df_temp = pd.read_html(driver.page_source)[0] #Pegar a table
        df = df.append(df_temp, ignore_index=True) #Atualizar df
    return df


In [4]:
def combinacao_3_caracteres(n):
    lista_caracteres = []
    for i in range(3,4,1):
        for j in range(0,n,1):
            for k in range(0,n,1):
                temp_name = (string.ascii_lowercase[i] + string.ascii_lowercase[j] + string.ascii_lowercase[k]) 
                lista_caracteres.append(temp_name)
    return lista_caracteres
combinacao_3_caracteres(26)

['daa',
 'dab',
 'dac',
 'dad',
 'dae',
 'daf',
 'dag',
 'dah',
 'dai',
 'daj',
 'dak',
 'dal',
 'dam',
 'dan',
 'dao',
 'dap',
 'daq',
 'dar',
 'das',
 'dat',
 'dau',
 'dav',
 'daw',
 'dax',
 'day',
 'daz',
 'dba',
 'dbb',
 'dbc',
 'dbd',
 'dbe',
 'dbf',
 'dbg',
 'dbh',
 'dbi',
 'dbj',
 'dbk',
 'dbl',
 'dbm',
 'dbn',
 'dbo',
 'dbp',
 'dbq',
 'dbr',
 'dbs',
 'dbt',
 'dbu',
 'dbv',
 'dbw',
 'dbx',
 'dby',
 'dbz',
 'dca',
 'dcb',
 'dcc',
 'dcd',
 'dce',
 'dcf',
 'dcg',
 'dch',
 'dci',
 'dcj',
 'dck',
 'dcl',
 'dcm',
 'dcn',
 'dco',
 'dcp',
 'dcq',
 'dcr',
 'dcs',
 'dct',
 'dcu',
 'dcv',
 'dcw',
 'dcx',
 'dcy',
 'dcz',
 'dda',
 'ddb',
 'ddc',
 'ddd',
 'dde',
 'ddf',
 'ddg',
 'ddh',
 'ddi',
 'ddj',
 'ddk',
 'ddl',
 'ddm',
 'ddn',
 'ddo',
 'ddp',
 'ddq',
 'ddr',
 'dds',
 'ddt',
 'ddu',
 'ddv',
 'ddw',
 'ddx',
 'ddy',
 'ddz',
 'dea',
 'deb',
 'dec',
 'ded',
 'dee',
 'def',
 'deg',
 'deh',
 'dei',
 'dej',
 'dek',
 'del',
 'dem',
 'den',
 'deo',
 'dep',
 'deq',
 'der',
 'des',
 'det',
 'deu',


In [10]:
df_table = pd.DataFrame(columns=['Sobrenome', 'Fonte', 'PROVINCIA', 'Comune',
                                 'DATA DE NASCIMENTO', 'Paternidade', 'Cônjuge',
                                 'Data deCasamento', 'MATRIMÔNIO','CHEGADA  AO BRASIL'])

download_path =  os.path.abspath(os.getcwd()) + r"\downloads\\" #Caminho da pasta de Download
url_login= "http://www.emigrazioneveneta.com/" #Login do EGS
backup_path = r"C:\Users\John Melati\Documents\GitHub\web_scraping_emigrazione_veneta\\"

try:
    driver = build_driver(download_path)
    login(driver, url_login)
    element_click_xpath("//html/body/div[1]/main/section[1]/div/div[2]/a") #Search Button
    
    lista_caracteres = combinacao_3_caracteres(26)
    for palavra in lista_caracteres:
        filter_project(palavra) #Filter Project

        texto = element_find_xpath("//html/body/div[1]/main/div[2]/form/p")
        if texto != "Nenhum resultado encontrado":
            print("Existem Registros")
            num_registros = int((element_find_xpath("//html/body/div[1]/main/div[2]/form/p")).split()[-1])
            print(num_registros)
            df_table = navegar_nas_paginas(palavra, num_registros, df_table)
            
        else:
            print("Vazio")
        
except Exception as i:
    driver.quit()
    print("Ocorreu uma exceção")  
    print(i)
    
time2=datetime.datetime.now()
print('Executado em: ',time2-time1)

Ligando o Chrome Remoto...


C:\Users\JOHNME~1\AppData\Local\Temp/ipykernel_21308/918995241.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\John Melati\Documents\GitHub\web_scraping_emigrazione_veneta\chromedriver.exe", options=options)


Ok
Entrando na página de login...
Ok


C:\Users\JOHNME~1\AppData\Local\Temp/ipykernel_21308/918995241.py:68: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  click_box = driver.find_element_by_xpath(xpath)


click ok
fill project


C:\Users\JOHNME~1\AppData\Local\Temp/ipykernel_21308/918995241.py:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  project_box = driver.find_element_by_xpath("//html/body/div[1]/main/div[2]/form/div/div[2]/input")


ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Existem Registros
get text ok
1
0
http://www.emigrazioneveneta.com/busca?searchword=eav&searchphrase=all&limitstart=0
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
g

ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok
Vazio
fill project
ok
get text ok

In [11]:
df_table

,Sobrenome,Fonte,PROVINCIA,Comune,DATA DE NASCIMENTO,Paternidade,Cônjuge,Data deCasamento,MATRIMÔNIO,CHEGADA AO BRASIL
0,Eavvidi Serafino,R.L.,---,Informazioni Conosciute,1876,* enteado,---,---,---,---
1,Ebbiotto Brigida,F.S.,---,Luogo Conosciuto,1857,di n.n.,Vascon Gio Batta,02-01-1879,Luogo Conosciuto,20-11-1892 S.Paolo *
2,Ebellindi Enrico Leonardo,R.P.,---,Luogo Conosciuto,12-07-1883,di n.n.,* con Brocco Giuseppe,---,---,27-12-1888 Vittoria *
3,Eberle Abramo Giuseppe,RGS1,---,Luogo Conosciuto,02-04-1880,di Giacomo e Zanrosso Giacoma,---,---,---,29-12-1884 Rio de Janeiro *
4,Eberle Domenico,RGS1,---,Luogo Conosciuto,14-07-1873,di Giovanni e Ruaro Luigia,---,---,---,Rio Grande do Sul *
...,...,...,...,...,...,...,...,...,...,...
62,Emerici Gino,R.P.,---,Luogo Conosciuto,14-04-1899,di Giustiniano e Gumiero Eugenia,---,---,---,07-12-1899 S.Paolo *
63,Emerici Giustiniano,P.A.,---,Luogo Conosciuto,01-02-1867,di n.n.,Gumiero Eugenia,09-06-1888,Luogo Conosciuto,07-12-1899 S.Paolo *
64,Emerici Luigi,R.P.,---,Luogo Conosciuto,22-01-1894,di Giustiniano e Gumiero Eugenia,---,---,---,07-12-1899 S.Paolo *
65,Emerici Modesta Pasqua,R.P.,---,Luogo Conosciuto,17-06-1888,di Giustiniano e Gumiero Eugenia,---,---,---,07-12-1899 S.Paolo *


In [12]:
def save_df_to_folder(df, download_path):
    '''
    Esta função realiza o backup (cópia) do arquivo para a pasta de backup.
    
    args:
    
    df_egs: dataframe
    backup_path: caminho onde encontra-se a pasta backup.
    
    '''
    print("saving backup")
    timestr = time.strftime("%Y_%m_%d-%H_%M_%S") 
    df.to_excel(download_path + f"df_E_{timestr}.xlsx", index=False)
    print("Ok")
    
save_df_to_folder(df_table, download_path)

saving backup
Ok
